In [242]:
import pandas as pd
import numpy as np
import re

In [5]:
from os import listdir
from os.path import isfile, join
mypath = '/home/michael/fantasy_basketball_project/scraper/data/dailyteams/'
files = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [10]:
df = pd.DataFrame()
for file in files:
    newDf = pd.read_csv(mypath+file)
    df = df.append(newDf)

In [13]:
df.shape

(14742, 16)

In [67]:
df.head()

,pos,player_team_status_pos,opp,result,MIN,REB,AST,STL,BLK,PTS,PR15,%ROST,+/-,team_id,scoring_period,date
0,C,Nikola VucevicOrlC,@Cle,L 93-107,36.0,13.0,6.0,2.0,2.0,28.0,4.36,97.8,-1.4,1,139,2019-03-03
1,G,Eric BledsoeMilPG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.63,96.5,0.0,1,139,2019-03-03
2,G,Kris DunnOChiPG,Atl,L 118-123,27.0,4.0,6.0,2.0,0.0,14.0,-5.20,50.8,-1.1,1,139,2019-03-03
3,G,Empty,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,139,2019-03-03
4,F,"Jerami GrantOKCSF, PF",Mem,W 99-95,37.0,7.0,5.0,0.0,1.0,13.0,4.03,71.8,1.9,1,139,2019-03-03


In [68]:
max(df['scoring_period'])

177

In [258]:
playedDf = df[pd.notnull(df['MIN'])]
playedDf = playedDf[playedDf['pos'] != 'Bench']
playedDf['name'] = playedDf['player_team_status_pos'].map(lambda x: "".join(re.split("([A-Z][^A-Z]*)", x)[1:5]))

In [259]:
playedDf.head()

,pos,player_team_status_pos,opp,result,MIN,REB,AST,STL,BLK,PTS,PR15,%ROST,+/-,team_id,scoring_period,date,name
0,C,Nikola VucevicOrlC,@Cle,L 93-107,36.0,13.0,6.0,2.0,2.0,28.0,4.36,97.8,-1.4,1,139,2019-03-03,Nikola Vucevic
2,G,Kris DunnOChiPG,Atl,L 118-123,27.0,4.0,6.0,2.0,0.0,14.0,-5.20,50.8,-1.1,1,139,2019-03-03,Kris Dunn
4,F,"Jerami GrantOKCSF, PF",Mem,W 99-95,37.0,7.0,5.0,0.0,1.0,13.0,4.03,71.8,1.9,1,139,2019-03-03,Jerami Grant
5,F,PJ TuckerHouSF,@Bos,W 115-104,40.0,8.0,1.0,1.0,1.0,9.0,0.81,6.4,-1.6,1,139,2019-03-03,PJ
17,G,Kemba WalkerFAPG,NaN,NaN,39.0,0.0,12.0,0.0,0.0,18.0,7.08,99.9,0.0,2,139,2019-03-03,Kemba Walker


In [260]:
posDf = playedDf.groupby(['scoring_period', 'team_id','pos'])[['date']].count()

In [261]:
cumulativeDf = posDf.loc[1]
totalPosDf = posDf.loc[1]
totalPosDf['period'] = 1

for period in list(set(playedDf['scoring_period']))[1:]:
    periodDf = posDf.loc[period]
    cumulativeDf = cumulativeDf.add(periodDf, fill_value=0)
    cumulativeDf['period'] = period
    totalPosDf = totalPosDf.append(cumulativeDf)

In [262]:
breachPeriodDf = totalPosDf.reset_index().set_index(['team_id'])

In [263]:
centerBreach = breachPeriodDf[breachPeriodDf['pos'] == 'C']
centerBreach = centerBreach[centerBreach['date'] == 82]
centerBreach = centerBreach.min(level='team_id')

In [264]:
forwardBreach = breachPeriodDf[breachPeriodDf['pos'] == 'F']
forwardBreach = forwardBreach[forwardBreach['date'] == 245]
forwardBreach = forwardBreach.min(level='team_id')

In [265]:
guardBreach = breachPeriodDf[breachPeriodDf['pos'] == 'G']
guardBreach = guardBreach[guardBreach['date'] == 245]
guardBreach = guardBreach.min(level='team_id')
guardBreach.loc[5] = ['G',247, 152]

In [266]:
utilBreach = breachPeriodDf[breachPeriodDf['pos'] == 'UTIL']
utilBreach = utilBreach[utilBreach['date'] == 245]
utilBreach = utilBreach.min(level='team_id')

In [267]:
periodDf = pd.concat([centerBreach, forwardBreach, guardBreach, utilBreach]).reset_index().set_index(['team_id','pos'])

In [268]:
periodDf.loc[1,'C']['period']

138.0

In [269]:
def validateSlot(team_id, position, scoring_period):
    try:
        validPeriod = periodDf.loc[team_id, position]['period']
    except:
        validPeriod = 177
    return scoring_period <= validPeriod

In [270]:
validPlayedDf = playedDf.copy(deep=True)
validPlayedDf['validSlot'] = validPlayedDf.apply(lambda row: validateSlot(row['team_id'], row['pos'], row['scoring_period']), axis=1)
validPlayedDf = validPlayedDf[validPlayedDf['validSlot']]

In [271]:
finalTotals = validPlayedDf.groupby(['team_id'], as_index=False)[['REB', 'AST', 'STL', 'BLK', 'PTS']].sum()
finalTotals['REBFPts'] = finalTotals['REB'].rank()
finalTotals['ASTFPts'] = finalTotals['AST'].rank()
finalTotals['STLFPts'] = finalTotals['STL'].rank()
finalTotals['BLKFPts'] = finalTotals['BLK'].rank()
finalTotals['PTSFPts'] = finalTotals['PTS'].rank()
finalTotals['TotalFPts'] = finalTotals['REBFPts'] + finalTotals['ASTFPts'] + finalTotals['STLFPts'] + finalTotals['BLKFPts'] + finalTotals['PTSFPts']

In [272]:
finalTotals.head(6)

,team_id,REB,AST,STL,BLK,PTS,REBFPts,ASTFPts,STLFPts,BLKFPts,PTSFPts,TotalFPts
0,1,6050.0,3623.0,959.0,697.0,13783.0,6.0,5.0,5.0,4.0,2.0,22.0
1,2,6022.0,3021.0,974.0,776.0,16746.0,5.0,1.0,6.0,6.0,6.0,24.0
2,3,5967.0,3865.0,909.0,735.0,15044.0,4.0,6.0,3.0,5.0,3.0,21.0
3,4,5960.0,3467.0,943.0,648.0,15245.0,3.0,4.0,4.0,2.0,4.0,17.0
4,5,5597.0,3228.0,849.0,689.0,15474.0,2.0,3.0,2.0,3.0,5.0,15.0
5,6,5023.0,3027.0,702.0,410.0,12534.0,1.0,2.0,1.0,1.0,1.0,6.0


In [273]:
validPlayedDf.groupby(['team_id','pos'])[['date']].count()

date
team_id pos       
1       C       82
        F      245
        G      245
        UTIL   245
2       C       82
        F      245
        G      245
        UTIL   245
3       C       82
        F      245
        G      245
        UTIL   245
4       C       82
        F      245
        G      245
        UTIL   245
5       C       80
        F      236
        G      247
        UTIL   245
6       C       67
        F      212
        G      212
        UTIL   218

In [274]:
validPlayedDf.to_pickle('/home/michael/fantasy_basketball_project/s1Validated.pkl')

In [241]:
finalTotals.to_pickle('/home/michael/fantasy_basketball_project/s1Summary.pkl')